In [1]:
from pathlib import Path
from functools import reduce
import pandas as pd
import numpy as np
import os
from datetime import datetime

from nanoHUB.application import Application
from nanoHUB.rfm.model import LastUpdateRecord, TempUserDescriptors, UserDescriptors

from sqlalchemy import select
from sqlalchemy.orm import Session

application = Application.get_instance()

nanoHUB - Serving Students, Researchers & Instructors


In [2]:
CACHE_DIR = Path(Path(os.getenv('APP_DIR')), '.cache')
NANOHUB_CACHE = Path(CACHE_DIR, 'nanohub')
NANOHUB_METRICS_CACHE = Path(CACHE_DIR, 'nanohub_metrics')

In [5]:
df = pd.read_csv(Path(CACHE_DIR, 'toolevents_rfm.csv'))

In [6]:
display(df)

,Unnamed: 0,user,first_start_date,last_start_date,first_finish_date,last_finish_date,job_count,superjob_count,job_event_days,superjob_event_days,last_superjob_start_date,toolevents__lifetime
0,0,NaN,2011-11-22 09:44:56,2013-05-28 15:46:14,2005-11-11 00:00:00,2013-05-28 15:46:17,885,NaN,10,NaN,NaN,553
1,1,0.yao.yuan,2017-03-08 10:15:23,2017-03-09 05:09:22,2017-03-08 10:16:34,2017-03-09 07:22:59,15,NaN,2,NaN,NaN,0
2,2,008dilip,2016-09-28 20:27:13,2016-10-12 23:50:56,2016-09-28 20:32:31,2016-10-13 00:08:08,14,NaN,5,NaN,NaN,14
3,3,00ff,2013-11-19 00:53:23,2013-11-19 00:58:56,2013-11-19 01:01:19,2013-11-19 01:04:21,2,NaN,1,NaN,NaN,0
4,4,00thamizharasi00,2017-03-20 09:46:36,2017-03-20 09:46:36,2017-03-20 09:46:37,2017-03-20 09:46:37,1,NaN,1,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
155302,155302,zzz121243,2021-05-04 19:04:27,2021-05-04 19:21:22,2021-05-04 19:18:31,2021-05-05 19:37:43,17,NaN,1,NaN,NaN,1
155303,155303,zzz1ttt,2015-10-21 21:42:30,2015-12-17 06:47:37,2015-10-21 21:44:26,2015-12-17 09:14:10,60,NaN,6,NaN,NaN,56
155304,155304,zzz777,2016-08-08 15:35:26,2016-08-08 16:05:25,2016-08-08 15:35:56,2016-08-08 16:07:32,9,NaN,1,NaN,NaN,0
155305,155305,zzzstas,2016-07-13 00:40:03,2016-07-13 06:33:44,2016-07-13 00:53:35,2016-07-13 08:23:48,40,NaN,1,NaN,NaN,0


In [9]:
dfc = df[(df.first_start_date >= '2005-11-12')]
display(dfc)

,Unnamed: 0,user,first_start_date,last_start_date,first_finish_date,last_finish_date,job_count,superjob_count,job_event_days,superjob_event_days,last_superjob_start_date,toolevents__lifetime
0,0,NaN,2011-11-22 09:44:56,2013-05-28 15:46:14,2005-11-11 00:00:00,2013-05-28 15:46:17,885,NaN,10,NaN,NaN,553
1,1,0.yao.yuan,2017-03-08 10:15:23,2017-03-09 05:09:22,2017-03-08 10:16:34,2017-03-09 07:22:59,15,NaN,2,NaN,NaN,0
2,2,008dilip,2016-09-28 20:27:13,2016-10-12 23:50:56,2016-09-28 20:32:31,2016-10-13 00:08:08,14,NaN,5,NaN,NaN,14
3,3,00ff,2013-11-19 00:53:23,2013-11-19 00:58:56,2013-11-19 01:01:19,2013-11-19 01:04:21,2,NaN,1,NaN,NaN,0
4,4,00thamizharasi00,2017-03-20 09:46:36,2017-03-20 09:46:36,2017-03-20 09:46:37,2017-03-20 09:46:37,1,NaN,1,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
155302,155302,zzz121243,2021-05-04 19:04:27,2021-05-04 19:21:22,2021-05-04 19:18:31,2021-05-05 19:37:43,17,NaN,1,NaN,NaN,1
155303,155303,zzz1ttt,2015-10-21 21:42:30,2015-12-17 06:47:37,2015-10-21 21:44:26,2015-12-17 09:14:10,60,NaN,6,NaN,NaN,56
155304,155304,zzz777,2016-08-08 15:35:26,2016-08-08 16:05:25,2016-08-08 15:35:56,2016-08-08 16:07:32,9,NaN,1,NaN,NaN,0
155305,155305,zzzstas,2016-07-13 00:40:03,2016-07-13 06:33:44,2016-07-13 00:53:35,2016-07-13 08:23:48,40,NaN,1,NaN,NaN,0


,Unnamed: 0,user,first_start_date,last_start_date,first_finish_date,last_finish_date,job_count,superjob_count,job_event_days,superjob_event_days,last_superjob_start_date,toolevents__lifetime
13,13,02junho,2016-05-04 04:22:01,2018-09-03 10:02:11,2016-05-04 04:25:05,2018-09-04 10:04:03,342,2.0,37,2.0,2018-08-04 10:01:18,853
28,28,082080112abk,2011-02-22 13:30:53,2013-08-13 02:11:08,2011-02-22 13:31:52,2013-08-14 02:01:31,1047,11.0,57,10.0,2011-12-06 23:32:21,903
43,43,0kt0pus.dr34m,2020-12-25 10:11:32,2020-12-25 10:29:59,2020-12-25 10:29:47,2020-12-26 10:30:56,4,1.0,1,1.0,2020-12-25 10:13:56,1
126,126,14zac2,2021-07-12 05:19:48,2021-07-19 06:09:53,2021-07-12 05:20:03,2021-07-19 06:13:34,9,3.0,2,2.0,2021-07-19 06:09:53,7
131,131,150_cm,2018-11-23 02:39:19,2018-11-24 21:53:01,2018-11-23 04:02:27,2018-11-25 13:34:37,133,11.0,2,1.0,2018-11-24 21:53:01,2
...,...,...,...,...,...,...,...,...,...,...,...,...
155231,155231,zzaa,2008-11-27 13:25:13,2012-01-07 07:47:24,2008-11-27 20:32:05,2012-01-07 07:47:24,155,1.0,32,1.0,2009-05-06 06:57:21,1135
155242,155242,zzeng,2017-09-21 09:54:38,2017-10-11 10:31:46,2017-09-21 10:23:18,2017-10-13 06:41:29,47,4.0,7,4.0,2017-10-11 10:03:17,21
155254,155254,zzhan163,2020-04-16 08:35:33,2020-05-07 08:32:20,2020-04-17 08:41:12,2020-05-08 09:15:40,25872,1.0,8,1.0,2020-04-20 12:36:06,22
155262,155262,zzhiheng,2016-10-31 16:43:37,2016-10-31 17:07:02,2016-10-31 16:49:02,2016-10-31 18:06:26,16,5.0,1,1.0,2016-10-31 17:07:02,0
